# SageMaker Pipelines Customer Churn Prediction

Amazon SageMaker Model Building Pipelines offers machine learning (ML) application developers and operations engineers the ability to orchestrate SageMaker jobs and author reproducible ML pipelines. It also enables them to deploy custom-build models for inference in real-time with low latency, run offline inferences with Batch Transform, and track lineage of artifacts. They can institute sound operational practices in deploying and monitoring production workflows, deploying model artifacts, and tracking artifact lineage through a simple interface, adhering to safety and best practice paradigms for ML application development.

The SageMaker Pipelines service supports a SageMaker Pipeline domain specific language (DSL), which is a declarative JSON specification. This DSL defines a directed acyclic graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that engineers and scientists are already familiar with.

## SageMaker Pipelines

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

* Pipelines - A DAG of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional execution steps - A step that provides conditional execution of branches in a pipeline.
* Register model steps - A step that creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Create model steps - A step that creates a model for use in transform steps or later publication as an endpoint.
* Clarify steps - A ClarifyCheck step that conduct model explainability check which launches a processing job that runs the SageMaker Clarify prebuilt container.
* Parametrized Pipeline executions - Enables variation in pipeline executions according to specified parameters.

## Notebook Overview

This notebook shows how to:

* Define a set of Pipeline parameters that can be used to parametrize a SageMaker Pipeline.
* Define a Processing step that extracts data from feature store to create the train, validation and test data sets.
* Define a Training step that trains a model on the preprocessed train data set.
* Define a Processing step that evaluates the trained model's performance on the test dataset.
* Define a Create Model step that creates a model from the model artifacts used in training.
* Define a Clarify check step that performs model explainability check.
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes other steps.
* Define a Register Model step that creates a model package from the estimator and model artifacts used to train the model.
* Define and create a Pipeline definition in a DAG, with the defined parameters and steps.
* Start a Pipeline execution and wait for execution to complete.
* Download the model evaluation report from the S3 bucket for examination.


## A SageMaker Pipeline

The pipeline that you create follows a typical machine learning (ML) application pattern of preprocessing, training, evaluation, model creation, and model registration:

![A typical ML Application pipeline](./img/pipeline-full.png)

<div class="alert alert-info">💡 <strong> Warning </strong>
  Please make sure to run the <strong>contact-center-data.ipynb</strong> notebook to prepare data for the pipeline
</div>


## Preparation


Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
!pip install graphviz -q

In [ ]:
import json
import os

import boto3
import sagemaker
import sagemaker.session
from sagemaker import get_execution_role

from typing import Dict

import sagemaker
from sagemaker.dataset_definition.inputs import (
    AthenaDatasetDefinition,
    DatasetDefinition,
)
from sagemaker.workflow.clarify_check_step import (
    DataBiasCheckConfig,
    ClarifyCheckStep,
    ModelBiasCheckConfig,
    ModelExplainabilityCheckConfig,
    ClarifyCheckConfig,
)
from sagemaker.clarify import (
    DataConfig,
    BiasConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import TrainingInput
from sagemaker.lambda_helper import Lambda
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.clarify_check_step import ClarifyCheckStep, DataBiasCheckConfig
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import (
    ParameterFloat,
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.step_collections import EstimatorTransformer, RegisterModel
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, Step, TrainingStep
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.functions import JsonGet
from sagemaker.utils import name_from_base

import utilities as utl
import uuid

In [ ]:
role = get_execution_role()

region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()

bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-customer-churn-connect'
base_job_prefix = 'Demo-xgboost-churn-connect'

s3_client = boto3.client("s3")

fg_name = <YOUR FEATURE GROUP NAME>  # Make sure to update the feature group name based on the feature group created in the previous step

### Define Parameters to Parametrize Pipeline Execution

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `train_instance_count` - The instance count of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes ( "PendingManualApproval" is the default).
* `model_output` - The S3 bucket URI location of the model output path

![Define Parameters](./img/pipeline-1.png)

**You must have run the previous sequential notebooks to retrieve variables using the StoreMagic command.**

In [ ]:
# parameters for pipeline execution

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)

train_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1,
)
train_instance_type = ParameterString(
    name="TrainingInstance",
    default_value="ml.m5.xlarge",
)


model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval",
    enum_values=[
        "PendingManualApproval",
        "Approved",
    ],
)
model_output = ParameterString(
    name="ModelOutputUrl",
    default_value=f"s3://{bucket}/{prefix}/model",
)


### Define a Processing Step for dataset creation from the SageMaker Feature Store

First, develop a preprocessing script that is specified in the Processing step.

In this example, we have already created the `create_dataset.py`, which contains the dataset creation script, and stored the path to the script. You can use the following cell to load the stored value.

The Processing step executes the script on the input data. The Training step uses the preprocessed training features and labels to train a model. The Evaluation step uses the trained model and preprocessed test features and labels to evaluate the model.

![Define a Processing Step for Feature Engineering](./img/pipeline-2.png)

Next, create an instance of an `SKLearnProcessor` processor and use that in our `ProcessingStep`.

You also specify the `framework_version` to use throughout this notebook.

Note the `processing_instance_type` and `processing_instance_count` parameters used by the processor instance.

When creating a [Processing job](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateProcessingJob.html), one needs to specify the `ProcessingInputs` parameter which tell the SageMaker service where to get the input data. If the data is already available on S3, we can use the S3Input to define the inputs for the processing job. However, in our example, the data is stored in the offline Feature Store, we can use the [DatasetDefinition](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DatasetDefinition.html) which supports the data sources like S3 which can be queried via Athena and Redshift. We use the [AthenaDatasetDefinition](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_AthenaDatasetDefinition.html) option, it executes SQL queries and generate datasets to S3 which will be available as the inputs of the processing job.

In [ ]:
# processing step for dataset creation
dataset_dict = {
    "customers_fg_name": fg_name, # fill your fg name that was created in your account, for example "fg-contact-center-data-352ebe3e"
    "label_name": "churn_true",
    "features_names": [
        'account_length', 
        'vmail_message', 
        'day_mins',
        'day_calls', 
        'eve_mins', 
        'eve_calls', 
        'night_mins', 
        'night_calls',
        'intl_mins', 
        'intl_calls', 
        'custserv_calls', 
        'sentiment',
        'pastsenti_nut', 
        'pastsenti_pos', 
        'pastsenti_neg', 
        'mth_remain',
        'int_l_plan_no', 
        'int_l_plan_yes', 
        'vmail_plan_no', 
        'vmail_plan_yes',
    ],
}

customers_fg_info = utl.get_fg_info(
    dataset_dict["customers_fg_name"],
    sagemaker_session=sagemaker_session,
)

label_name = dataset_dict["label_name"]
features_names = dataset_dict["features_names"]
training_columns = [label_name] + features_names
training_columns_string = ", ".join(f'"{c}"' for c in training_columns)

query_string = f"""SELECT DISTINCT {training_columns_string}
    FROM "{customers_fg_info.table_name}" 
"""
print(query_string)

In [ ]:
athena_data_path = "/opt/ml/processing/athena"
script_path = './scripts/create_dataset.py'

create_dataset_processor = SKLearnProcessor(
    framework_version="0.23-1",
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name=f"{base_job_prefix}-create-dataset",
    sagemaker_session=sagemaker_session,
    role=role,
)

In [ ]:
data_sources = [
    ProcessingInput(
        input_name="athena_dataset",
        dataset_definition=DatasetDefinition(
            local_path=athena_data_path,
            data_distribution_type="FullyReplicated",
            athena_dataset_definition=AthenaDatasetDefinition(
                catalog=customers_fg_info.catalog,
                database=customers_fg_info.database,
                query_string=query_string,
                output_s3_uri=Join(
                    on="/",
                    values=[
                        "s3:/",
                        bucket,
                        prefix,
                        ExecutionVariables.PIPELINE_EXECUTION_ID,
                        "raw_dataset",
                    ],
                ),
                output_format="PARQUET",
            ),
        ),
    )
]

Finally, use the processor instance to construct a `ProcessingStep`, along with the input and output channels, and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` is the input data used in the step. This input data is used by the processor instance when it is run.

Also, note the `"train"`, `"validation"` and `"test"` named channels specified in the output configuration for the processing job. Note that we also creates a baseline csv file that will be used in later clarify step for model explainability check. Step `Properties` can be used in subsequent steps and resolve to their runtime values at execution. Specifically, this usage is called out when you define the training step.

In [ ]:
create_dataset_step = ProcessingStep(
    name="CreateDataSet",
    processor=create_dataset_processor,
    inputs=data_sources,
    outputs=[
        ProcessingOutput(
            output_name="train_data",
            source="/opt/ml/processing/output/train",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    bucket,
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "train_dataset",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="validation_data",
            source="/opt/ml/processing/output/validation",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    bucket,
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "validation_dataset",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="test_data",
            source="/opt/ml/processing/output/test",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    bucket,
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "test_dataset",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="baseline_data",
            source="/opt/ml/processing/output/baseline",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    bucket,
                    prefix,
                    "baseline",
                ],
            ),
        ),
    ],
    job_arguments=[
        "--athena-data",
        athena_data_path,
    ],
    code=script_path,
)

### Define a Training Step to Train a Model

In this section, use Amazon SageMaker's [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train on this dataset. Configure an Estimator for the XGBoost algorithm and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_output` so that it can be hosted later. 

The model path where the models from training will be saved is also specified.

Note the `training_instance_type` parameter may be used in multiple places in the pipeline. In this case, the `training_instance_type` and `train_instance_count` is passed into the estimator.

![Define a Training Step to Train a Model](./img/pipeline-3.png)

In [ ]:
# training step for generating model artifacts
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
)
xgb_train = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    instance_type=train_instance_type,
    instance_count=train_instance_count,
    output_path=model_output,
    base_job_name=f"{base_job_prefix}-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set some hyper parameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_train.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
    eval_metric='auc'
)


Finally, use the estimator instance to construct a `TrainingStep` as well as the `properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that's executed when the pipeline invokes the pipeline execution. This is similar to an estimator's `fit` method in the Python SDK.

Pass in the `S3Uri` of the `"train"` output channel to the `TrainingStep`. Also, use the other `"validation"` output channel for model evaluation in the pipeline. The `properties` attribute of a Pipeline step matches the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [ ]:
step_train = TrainingStep(
    name="TrainModel",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=create_dataset_step.properties.ProcessingOutputConfig.Outputs[
                "train_data"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=create_dataset_step.properties.ProcessingOutputConfig.Outputs[
                "validation_data"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },

)

### Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including mae, mse, rmse and r2 metrics.
* Save the evaluation report to the evaluation directory.

![Define a Model Evaluation Step to Evaluate the Trained Model](./img/pipeline-4.png)

Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [ ]:
# processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=f"{base_job_prefix}/script-eval",
    sagemaker_session=sagemaker_session,
    role=role,
)


Use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Specifically, the `S3ModelArtifacts` from the `step_train` `properties` and the `S3Uri` of the `"test"` output channel of the `step_process` `properties` are passed into the inputs. The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [ ]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)
step_eval = ProcessingStep(
    name="EvaluateModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=create_dataset_step.properties.ProcessingOutputConfig.Outputs[
                "test_data"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation", source="/opt/ml/processing/evaluation"
        ),
    ],
    code="./scripts/evaluation.py",
    property_files=[evaluation_report],

)

### Define a Create Model Step to Create a Model

In order to perform batch transformation using the example model, create a SageMaker model. 

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

![Define a Create Model Step and Batch Transform to Process Data in Batch at Scale](./img/pipeline-5.png)

In [ ]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.model import Model

model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.xlarge",
)
step_create_model = CreateModelStep(
    name="CustomerChurnCreateModel",
    model=model,
    inputs=inputs,
)

### Clarify configuration

[Amazon SageMaker Clarify](https://aws.amazon.com/sagemaker/clarify/) helps improve your machine learning models by detecting potential bias and helping explain how these models make predictions. The fairness and explainability functionality provided by SageMaker Clarify takes a step towards enabling AWS customers to build trustworthy and understandable machine learning models.

### Explaining Predictions
There are expanding business needs and legislative regulations that require explanations of why a model made the decision it did. SageMaker Clarify uses [SHAP library](https://github.com/slundberg/shap) to explain the contribution that each input feature makes to the final decision. SageMaker Clarify uses a scalable and efficient implementation of [Kernel SHAP](https://github.com/slundberg/shap#model-agnostic-example-with-kernelexplainer-explains-any-function) with an option to use spark based parallelization with multiple processing instances.

As a contrastive explainability technique, SHAP values are calculated by evaluating the model on synthetic data generated against a baseline sample. The explanations of the same case can be different depending on the choices of this baseline sample.

We use the [mode](https://en.wikipedia.org/wiki/Mode_(statistics) statistic to create the baseline for categorical data and `mean` for numerical data. The mode is a good choice for categorical variables. This step is already done during the dataset creation step, please see details in the `create_dataset.py` file.

For more information on selecting informative vs non-informative baselines, see [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)

![Define Parameters](./img/pipeline-6.png)

#### Setup configurations for Clarify
Next, setup some more configurations to start the explainability analysis by Clarify. We need to set up the following:

1. SHAPConfig: to create the baseline. In this example, the mean_abs is the mean of absolute SHAP values for all instances, specified as the baseline
2. DataConfig: to provide some basic information about data I/O to SageMaker Clarify. We specify where to find the input dataset, where to store the output, the header names, and the dataset type.
3. ModelConfig: to specify information about the trained model here we re-use the model name created earlier

To know more about what these configurations mean for Clarify, check out the documentation here: https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html

In [ ]:
check_job_config = CheckJobConfig(
                role=role,
                instance_type="ml.m5.xlarge",
                instance_count=1,
                sagemaker_session=sagemaker_session,
)

# explainability output path
explainability_output_uri = "s3://{}/{}/clarify-explainability".format(bucket, prefix)

data_config = DataConfig(
                s3_data_input_path=create_dataset_step.properties.ProcessingOutputConfig.Outputs[
                    "train_data"
                ].S3Output.S3Uri,
                s3_output_path=explainability_output_uri,
                label=dataset_dict["label_name"],
                headers=[dataset_dict["label_name"]] + dataset_dict["features_names"],
                dataset_type="text/csv",
)
shap_config = SHAPConfig(
                baseline=f"s3://{bucket}/{prefix}/baseline/baseline.csv",
                num_samples=15,
                agg_method="mean_abs",
                save_local_shap_values=True,
)
model_config = ModelConfig(
                model_name=step_create_model.properties.ModelName,
                instance_type="ml.m5.xlarge",
                instance_count=1,
                accept_type="text/csv",
                content_type="text/csv",
)


Now create the model explainability check step. For more information about the ClarifyCheck step, please refer to the [doc here](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check).

In [ ]:
model_explainability_check_config = ModelExplainabilityCheckConfig(
    data_config=data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

model_explainability_check_step = ClarifyCheckStep(
    name="ModelExplainabilityCheckStep",
    clarify_check_config=model_explainability_check_config,
    check_job_config=check_job_config,
    skip_check=True,
    register_new_baseline=True,
)

### Define a Register Model Step to Create a Model Package

Use the estimator instance specified in the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

The construction of `RegisterModel` is similar to an estimator instance's `register` method in the Python SDK.

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Note that the specific model package group name provided in this notebook can be used in the model registry and CI/CD work with SageMaker Projects.

![Define Parameters](./img/pipeline-7.png)

In [ ]:
# register model step that will be conditionally executed
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
                "S3Uri"
            ]
        ),
        content_type="application/json",
    )
)
model_package_group_name = "CustomerChurnModelPackage"
step_register = RegisterModel(
    name="RegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.t2.large", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

### Define a Condition Step to Check Accuracy and Conditionally Register a Model in the Model Registry

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties. 

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `CreateModelStep` and `TransformStep` steps, and the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed, if the condition evaluates to `True`.

![Define a Condition Step to Check Accuracy and Conditionally Execute Steps](./img/pipeline-8.png)

In [ ]:
# condition step for evaluating model quality and branching execution
cond_lte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.roc_auc.value",
    ),
    right=0.95,
)
step_cond = ConditionStep(
    name="CheckEvaluation",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[],
)

### Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All of the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the _data dependency_ DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

![Define a Pipeline of Parameters, Steps, and Conditions](./img/pipeline-9.png)

In [ ]:
# pipeline instance
pipeline_name = f"Demo-customer-churn-pipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        processing_instance_type,
        train_instance_count,
        train_instance_type,
        model_approval_status,
        model_output
    ],
    steps=[create_dataset_step, step_train, step_eval, step_create_model, model_explainability_check_step, step_cond],
    sagemaker_session=sagemaker_session,
)

### Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The role passed in will be used by the Pipeline service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=role)

### Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution.

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
# Wait for the execution to complete.
execution.wait()

### Examining the Evalution

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [ ]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
pprint(json.loads(evaluation_json))

### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

## Clean Up

In [ ]:
boto_session = boto3.Session(region_name=region)
sagemaker_boto_client = boto_session.client("sagemaker")

# delete pipeline
sagemaker_boto_client.delete_pipeline(PipelineName=pipeline_name)

# delete feature group
sagemaker_boto_client.delete_feature_group(FeatureGroupName=fg_name)

# delete model package
model_packages = sagemaker_boto_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]
for mp in model_packages:
    sagemaker_boto_client.delete_model_package(ModelPackageName=mp["ModelPackageArn"])
    print(f"\nDeleted model package: {mp['ModelPackageArn']}")
    time.sleep(1)
sagemaker_boto_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

# delete s3 objects
delete_s3_objects = False # change this value to True if you would like to delete the s3 objects created by this project
if delete_s3_objects == True and bucket is not None:
    s3 = boto3.resource("s3")
    bucket = s3.Bucket(bucket)
    bucket.objects.filter(Prefix=f"{prefix}/").delete()
    print(f"\nDeleted contents of {bucket_name}/{prefix}")